<a href="https://colab.research.google.com/github/anuj-l22/PRML_Project/blob/main/PRML_Feat_ext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### The dataset was imported using !wget command and stored in lfw-deepfunneled.tgz and then extracted into lfw-deepfunneled

In [ ]:
!wget http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz

In [ ]:
!tar -xvzf lfw-deepfunneled.tgz

### Resnet Features

In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# Check if a GPU is available and set the device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Load pre-trained ResNet-50 model and move it to the device
resnet = models.resnet50(pretrained=True).to(device)
# Remove the last fully connected layer
resnet = nn.Sequential(*list(resnet.children())[:-1])
# Set the model to evaluation mode
resnet.eval()

# Define a function to extract features from an image
def extract_features(image_path, model, device):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = preprocess(image)
    # Add batch dimension and move the image to the device
    image = image.unsqueeze(0).to(device)
    # Extract features
    with torch.no_grad():
        features = model(image)
    # Remove the batch dimension and move the features to the CPU
    features = features.squeeze(0).cpu()
    return features

# Path to the LFW dataset
dataset_path = '/content/lfw-deepfunneled'

# Dictionary to store features
features_dict = {}

# Loop through all the images in the dataset
for person in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person)
    if os.path.isdir(person_path):
        for image_filename in os.listdir(person_path):
            image_path = os.path.join(person_path, image_filename)
            features = extract_features(image_path, resnet, device)
            features_dict[image_path] = features

# Save the extracted features to a file
torch.save(features_dict, 'extracted_resnet_features.pt')


### HOG

### Usage instructions for HoG extraction:
### Specify a start index and end index , for example for samples 2000 to 4000 , 4000 to 6000 , etc . This is done to make it computationally possible



In [ ]:
import os
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
import torch

def compute_hog(img):
    resized_img = resize(img, (128 * 4, 64 * 4))
    fd, _ = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                cells_per_block=(2, 2), visualize=True, channel_axis=-1)
    return fd

dataset_path = 'lfw-deepfunneled'
hog_features_dict = {}
counter = 0

image_files = []
for person in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person)
    if os.path.isdir(person_path):
        for image_filename in os.listdir(person_path):
            image_files.append(os.path.join(person_path, image_filename))

# Specify start and end indices here
start_index = 4000
end_index = 6000  # Adjust this as needed, e.g., 6000 for the second chunk

for image_path in image_files[start_index:end_index]:
    img = imread(image_path)
    fd = compute_hog(img)
    hog_features_dict[image_path] = fd
    counter += 1
    if counter % 100 == 0:
        print(f'{counter} HOG samples processed from index {start_index} to {end_index}.')

# Save the extracted HOG features to a file
save_filename = f'extracted_hog_features_{start_index}_{end_index}.pt'
torch.save(hog_features_dict, save_filename)
print(f'Features saved to {save_filename}')


### LBP

In [ ]:
import os
import cv2
import numpy as np
import torch

def get_pixel(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center:
            new_value = 1
    except:
        pass
    return new_value

def lbp_calculated_pixel(img, x, y):
    center = img[x][y]
    val_ar = []
    val_ar.append(get_pixel(img, center, x-1, y+1))     # top_right
    val_ar.append(get_pixel(img, center, x, y+1))       # right
    val_ar.append(get_pixel(img, center, x+1, y+1))     # bottom_right
    val_ar.append(get_pixel(img, center, x+1, y))       # bottom
    val_ar.append(get_pixel(img, center, x+1, y-1))     # bottom_left
    val_ar.append(get_pixel(img, center, x, y-1))       # left
    val_ar.append(get_pixel(img, center, x-1, y-1))     # top_left
    val_ar.append(get_pixel(img, center, x-1, y))       # top

    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val


def calcLBP(img):
    height, width, _ = img.shape
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    img_lbp = np.zeros((height, width), np.uint8)
    for i in range(0, height):
        for j in range(0, width):
            img_lbp[i, j] = lbp_calculated_pixel(img_gray, i, j)
    hist_lbp = cv2.calcHist([img_lbp], [0], None, [256], [0, 256])
    return hist_lbp.flatten()

# Path to the LFW dataset
dataset_path = 'content/lfw-deepfunneled'

# Dictionary to store LBP features
lbp_features_dict = {}

# Counter for processed samples
counter = 0

# Loop through all the images in the dataset
for person in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person)
    if os.path.isdir(person_path):
        for image_filename in os.listdir(person_path):
            image_path = os.path.join(person_path, image_filename)
            img = cv2.imread(image_path)
            lbp_feature = calcLBP(img)
            lbp_features_dict[image_path] = lbp_feature
            counter += 1
            if counter % 100 == 0:
                print(f'{counter} LBP samples processed.')

# Save the extracted LBP features to a file
torch.save(lbp_features_dict, 'extracted_lbp_features.pt')
